In [1]:
# Load packages
import os
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [2]:
# Load osm_building Features and calculate polygon centroids
home_path = os.getenv('HOME')
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

,osm_id,code,fclass,name,type,geometry
0,23453590,1500,building,None,None,"POLYGON ((3.28630 36.72337, 3.28680 36.72384, ..."
1,23453621,1500,building,None,industrial,"POLYGON ((3.28839 36.72555, 3.28945 36.72653, ..."
2,23453649,1500,building,None,industrial,"POLYGON ((3.29652 36.73153, 3.29695 36.73203, ..."
3,26224008,1500,building,Hilton Alger,hotel,"POLYGON ((3.15595 36.73836, 3.15634 36.73856, ..."
4,26224288,1500,building,Crystal Lounge,None,"POLYGON ((3.15561 36.73811, 3.15583 36.73823, ..."


In [3]:
# Set the correct CRS and reproject
osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
# Load the targeted dataset
pois_path = '../dz-datasets/accountants/'
pois = gpd.read_file(pois_path + 'pois_checked_in_out_wilayas.geojson')
pois.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code_wil,geometry
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,None,None,saidiabderaflr@gmail.com,33.618970,-7.130554,True,1,POINT (-7.13055 33.61897)
1,239,1511,BAKHOUIA,ABDERRAHMANE,"CITE 137 LOGTS, 1er MAI - ADRAR",Adrar,49966472,49966472,ab647200@yahoo.fr,28.017440,-0.264250,False,1,POINT (-0.26425 28.01744)
2,1418,1355,LAAFOU,AHMED,CITE 175 LOGTS ROUTETILILANE - ADRA R,Adrar,0499667OO,None,None,27.898056,-0.248333,False,1,POINT (-0.24833 27.89806)
3,61,1632,AHRES,HASSEN,RUE KADDOUR BELLITIM - ADRAR,Adrar,49964925,None,hassanahres@gmail.com,27.882773,-0.270904,False,1,POINT (-0.27090 27.88277)
4,327,1213,BELLAOUI,ABDERRAHMANE,CITE 05 JUILLET 1962 - ADRAR,Adrar,None,49951555,None,28.017440,-0.264250,False,1,POINT (-0.26425 28.01744)


In [5]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

pois = pois.set_crs("EPSG:4326")
pois = pois.to_crs("EPSG:3857")
pois.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# Replace DataFrame geometries with buffers and write it to a file
pois['buffer'] = pois.geometry.buffer(500)
buffers = pois[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
#gpd.GeoDataFrame(buffers).to_file(pois_path + "buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,1865,"POLYGON ((-793269.596 3977753.012, -793272.003..."
1,239,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
2,1418,"POLYGON ((-27144.337 3236127.002, -27146.744 3..."
3,61,"POLYGON ((-29656.840 3234202.120, -29659.247 3..."
4,327,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."


In [7]:
# Search for buildings that intersect the locations buffers
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')
res_intersection.count()

num         229595
osm_id      229595
code        229595
fclass      229595
name         27409
type         59297
geometry    229595
dtype: int64

In [8]:
res_intersection.head()

,num,osm_id,code,fclass,name,type,geometry
0,1418,433470111,1500,building,None,None,"POLYGON ((-27307.094 3236402.513, -27306.426 3..."
1,61,304696040,1500,building,Direction des Foret,None,"POLYGON ((-30027.197 3233895.509, -30008.217 3..."
2,61,432784883,1500,building,Direction des Foret,None,"POLYGON ((-30040.155 3233856.304, -30030.882 3..."
3,61,432784884,1500,building,None,None,"POLYGON ((-30022.266 3233831.130, -30013.382 3..."
4,61,432784886,1500,building,None,None,"POLYGON ((-30033.910 3233802.680, -30015.787 3..."


In [9]:
# Write intersected building into a GeoJSON file
res = res_intersection[['num', 'geometry']]
#res.to_file(pois_path + "intersection.geojson", driver='GeoJSON')
res.head()

,num,geometry
0,1418,"POLYGON ((-27307.094 3236402.513, -27306.426 3..."
1,61,"POLYGON ((-30027.197 3233895.509, -30008.217 3..."
2,61,"POLYGON ((-30040.155 3233856.304, -30030.882 3..."
3,61,"POLYGON ((-30022.266 3233831.130, -30013.382 3..."
4,61,"POLYGON ((-30033.910 3233802.680, -30015.787 3..."


In [10]:
# Group results of intersection and add a to_check column (if intersect then doesn't need to check)
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check_osm'] = False
res_grouped.count()

to_check_osm    1889
dtype: int64

In [11]:
# Merge with DataFrame and set geometry to centroids
pois_gdf = pois.merge(res_grouped, left_on='num', right_on='num', how='left')
pois_gdf.drop(columns=['buffer'], inplace=True)
pois_gdf.count()

num                2116
num_inscription    2116
nom                2116
prenom             2116
adresse            2116
wilaya             2116
tel                1389
fax                 994
email              1477
lat                2027
lon                2027
to_check           2116
code_wil           2116
geometry           2116
to_check_osm       1889
dtype: int64

In [12]:
# Update to_check column :if doesn't intersect then we need to check it manually
pois_gdf['to_check_osm'] = pois_gdf.apply(lambda row: row.to_check_osm != False, axis=1)
pois_gdf[pois_gdf['to_check_osm'] != False]

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code_wil,geometry,to_check_osm
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,None,None,saidiabderaflr@gmail.com,33.618970,-7.130554,True,1,POINT (-793769.596 3977753.012),True
1,239,1511,BAKHOUIA,ABDERRAHMANE,"CITE 137 LOGTS, 1er MAI - ADRAR",Adrar,49966472,49966472,ab647200@yahoo.fr,28.017440,-0.264250,False,1,POINT (-29416.142 3251172.790),True
4,327,1213,BELLAOUI,ABDERRAHMANE,CITE 05 JUILLET 1962 - ADRAR,Adrar,None,49951555,None,28.017440,-0.264250,False,1,POINT (-29416.142 3251172.790),True
5,1517,1189,MALKI,RACHID,RUE BELGHIT MOHAMMED N°20 - ADRAR,Adrar,49967999,None,malki2004@yahoo.fr,28.017440,-0.264250,False,1,POINT (-29416.142 3251172.790),True
6,1086,801,GRISI,MOHAMED,CIA DES FONCTIONNAIRES BT 05 N'C 06 - CHLEF,Chlef,27777822,None,None,36.417984,-79.650211,True,2,POINT (-8866620.874 4358288.772),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,1757,2027,OULAD HEDDAR,MAHAMMED,BP 5045 - GHARDAIA,Ghardaia,0298376OO,0298376OO,comptamoh@gmail.com,32.494374,3.644460,False,47,POINT (405699.431 3828381.296),True
2100,430,1877,BENKHELIFA,ALI,"26, AVENUE DU 1er NOVEMBRE -GHARDAIA",Ghardaia,29234702,29234703,alibenkhelifa@yahoo.fr,NaN,NaN,True,47,POINT (353693.063 3656147.148),True
2101,1033,273,FIHAKHIR,BACHIR,"T.E.M, RUE BELMOKHTAR SLIMANE - GHARDAIA",Ghardaia,29837281,29837281,None,32.494374,3.644460,False,47,POINT (405699.431 3828381.296),True
2106,1045,2182,GACE Ml,ATIKA,COLLECTIVITE 90 LOGEMENT 82 N'l6 OUED RHIOU -...,Relizane,None,None,atgaceml@gmall.com,35.977005,0.924750,False,48,POINT (102942.710 4297457.729),True


In [13]:
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['to_check_osm'] == True)   
final_gdf[final_gdf.to_check == True]

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code_wil,geometry,to_check_osm
0,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,None,None,saidiabderaflr@gmail.com,33.618970,-7.130554,True,1,POINT (-793769.596 3977753.012),True
1,239,1511,BAKHOUIA,ABDERRAHMANE,"CITE 137 LOGTS, 1er MAI - ADRAR",Adrar,49966472,49966472,ab647200@yahoo.fr,28.017440,-0.264250,True,1,POINT (-29416.142 3251172.790),True
4,327,1213,BELLAOUI,ABDERRAHMANE,CITE 05 JUILLET 1962 - ADRAR,Adrar,None,49951555,None,28.017440,-0.264250,True,1,POINT (-29416.142 3251172.790),True
5,1517,1189,MALKI,RACHID,RUE BELGHIT MOHAMMED N°20 - ADRAR,Adrar,49967999,None,malki2004@yahoo.fr,28.017440,-0.264250,True,1,POINT (-29416.142 3251172.790),True
6,1086,801,GRISI,MOHAMED,CIA DES FONCTIONNAIRES BT 05 N'C 06 - CHLEF,Chlef,27777822,None,None,36.417984,-79.650211,True,2,POINT (-8866620.874 4358288.772),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,430,1877,BENKHELIFA,ALI,"26, AVENUE DU 1er NOVEMBRE -GHARDAIA",Ghardaia,29234702,29234703,alibenkhelifa@yahoo.fr,NaN,NaN,True,47,POINT (353693.063 3656147.148),True
2101,1033,273,FIHAKHIR,BACHIR,"T.E.M, RUE BELMOKHTAR SLIMANE - GHARDAIA",Ghardaia,29837281,29837281,None,32.494374,3.644460,True,47,POINT (405699.431 3828381.296),True
2106,1045,2182,GACE Ml,ATIKA,COLLECTIVITE 90 LOGEMENT 82 N'l6 OUED RHIOU -...,Relizane,None,None,atgaceml@gmall.com,35.977005,0.924750,True,48,POINT (102942.710 4297457.729),True
2108,1130,1666,HADOAD,AHMED,RUE HAMANI BELOJILALI - GUELIZANE,Relizane,46718057,None,None,36.770662,3.056956,True,48,POINT (340298.785 4407188.097),False


In [14]:
# Drop unecessary columns
final_gdf = final_gdf.drop(columns=['to_check_osm'], axis=1)

In [15]:
# Reproject to geograĥic and Write to a GeoPackage file
final_gdf = final_gdf.set_crs("EPSG:3857")
final_gdf = final_gdf.to_crs("EPSG:4326")
final_gdf.to_file(pois_path + "pois_to_check.geojson", driver='GeoJSON')